## How to import the functions to read the labJack T7-Pro
Uses 9 analog inputs (AINs) to read the data at 200 Hz.
This notebook reads the accelerometers while taking an image
for mount tracking purposes.  To run it, you will first have to do the following:

(1) pip install labjack-ljm.  This will build the labJack Python code
    in your local directory at ~/.local/lib/python3.8/site-packages/labjack \
(2) In your ~/notebooks/.user_setups file, add the following lines: \
      export LD_LIBRARY_PATH=\\$LD_LIBRARY_PATH:/scratch/lib \
      export PYTHONPATH=\\$PYTHONPATH:/scratch/cslage/labJack

Craig Lage - Mar 10, 2022

In [ ]:
import sys, time, datetime, asyncio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from labjack_functions import LabJackFunctions

In [ ]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
#atcs = ATCS(domain)
latiss = LATISS(domain)
#await asyncio.gather(atcs.start_task, latiss.start_task)
await asyncio.gather(latiss.start_task)

In [ ]:
# enable components
#wait atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
# Take 5 biases to make sure things are working
# Added wait to stop killing the recent images
for i in range(5):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
test = await latiss.take_object(5.0, 1)
print(test)

In [ ]:
lj = LabJackFunctions(latiss)

In [ ]:
lj.setTimeout(60.0)

In [ ]:
# Take one 5 second image with accelerometer data
await lj.take_object_with_accel_data(5.0, filter='empty_1', grating='empty_1')

In [ ]:
# Take 1800 seconds of accelerometer data not tied to an image
await lj.take_accel_data(28800)

In [ ]:
lj.close()

In [ ]:
await latiss.standby()